## Поиск по артисту

In [8]:
import requests, json, urllib.parse
base_url = 'https://itunes.apple.com/search'

artist = 'Нервы'

params = {
    'term': artist,
    'limit': 200,    
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

## Первичная очистка данных

In [9]:
data = map(lambda x: {
    'artistName': x['artistName'],
    'kind' : x['kind'],
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionName'],
    'trackName': x['trackName'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'trackCount': x['trackCount'],
    'trackTimeMillis': x['trackTimeMillis'],
    'primaryGenreName': x['primaryGenreName']
}, data)
data = list(data)

## Функции

In [10]:
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}:Hours,{minutes}:Minutes,{seconds}:Seconds"

def filter_list(items, value):
    has_more = items.count(value) > 1
    return [x for x in items if not (x == value and has_more)]

## Сбор нужной статистики

In [11]:
import statistics

countOfSongs = len(list(map(lambda x: x['trackName'], data)))
dataGenre = (list((map(lambda x: x['primaryGenreName'], data))))
dataAlbums = len(list(set(map(lambda x: x['collectionId'], data))))
songsDuration = sum(list(map(lambda x : x['trackTimeMillis'],data)))

Average_time = statistics.mean(list(map(lambda x : x['trackTimeMillis'],data)))
median_time = list(map(lambda x : x['trackTimeMillis'],data))
median_time.sort()
median_time = statistics.median(median_time)

songsCost = sum(filter_list(list(map(lambda x: x['trackPrice'], data)),-1))
Average_cost = statistics.mean(filter_list(list(map(lambda x: x['trackPrice'], data)),-1))
median_cost = filter_list(list(map(lambda x: x['trackPrice'], data)),-1)
median_cost.sort()
median_cost = statistics.median(median_cost)

time_extractor = lambda x: x['trackTimeMillis']

longest_song = max(data, key=time_extractor)
shortest_song = min(data, key=time_extractor)

track_price_extractor = lambda x: x['trackPrice']

dearest_song = max(data, key=track_price_extractor)

сheapest_song = min(data, key=track_price_extractor)

## Представление всех данных по артисту

In [12]:
print('Artist:',artist)

print('Number of songs:',countOfSongs)

print('Popular genre:','Rock -',dataGenre.count('Rock'))

print('Album count:',dataAlbums)

print('Total duration of songs:',convert_millis(songsDuration))

print('Average duration of songs:',convert_millis(Average_time))

print('Median duration of songs:',convert_millis(median_time))

print('Longest song:', longest_song['trackName'], convert_millis(longest_song['trackTimeMillis']))

print('Shortest song:', shortest_song['trackName'], convert_millis(shortest_song['trackTimeMillis']))

print('Total cost of songs ≈ ',round(songsCost),'$')

print('Average cost of songs:',Average_cost,'$')

print('Median cost of songs:',median_cost,'$')

print('The most expensive song:', dearest_song['trackName'],'Trackrpice:',dearest_song['trackPrice'])

if сheapest_song['trackPrice'] == -1 : 
    print('Сheapest song:', сheapest_song['trackName'],'Track price:','Free')
else:
    print('Сheapest song:', сheapest_song['trackName'],'Track price:',сheapest_song['trackPrice'])

Artist: Нервы
Number of songs: 194
Popular genre: Rock - 163
Album count: 25
Total duration of songs: 11:Hours,56:Minutes,17:Seconds
Average duration of songs: 0.0:Hours,3.0:Minutes,41.0:Seconds
Median duration of songs: 0.0:Hours,3.0:Minutes,34.0:Seconds
Longest song: Лампами 0:Hours,8:Minutes,48:Seconds
Shortest song: Не хочу 0:Hours,1:Minutes,35:Seconds
Total cost of songs ≈  214 $
Average cost of songs: 1.1044329896907217 $
Median cost of songs: 0.99 $
The most expensive song: Кофе мой друг Trackrpice: 1.29
Сheapest song: Слишком влюблён Track price: 0.99


## Группировка песен в альбомы

In [13]:
from itertools import groupby

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

del sorted_by_album


## Сбор данных по альбому

In [14]:
album_name = album_names[1550772032]
album_values= list(album_groups[1550772032])

album_songs_count = len(list(set(map(lambda x: x['trackId'], album_values))))
dataGenre_Album = (list((map(lambda x: x['primaryGenreName'], album_values))))
releaseDate_Album = list(set(map(lambda x: x['releaseDate'], album_values)))

songsDuration_Album = sum(list(map(lambda x : x['trackTimeMillis'],album_values)))
Average_time_Album = statistics.mean(list(map(lambda x : x['trackTimeMillis'],album_values)))
median_time_Album = list(map(lambda x : x['trackTimeMillis'],album_values))
median_time_Album.sort()
median_time_Album = statistics.median(median_time_Album)

longest_song_Album = max(album_values, key=time_extractor)
shortest_song_Album = min(album_values, key=time_extractor)

dearest_song_Album = max(album_values, key=track_price_extractor)
сheapest_song_Album = min(album_values, key=track_price_extractor)

songsCost_Album = sum(filter_list(list(map(lambda x: x['trackPrice'], album_values)),-1))
Average_cost_Album = statistics.mean(filter_list(list(map(lambda x: x['trackPrice'], album_values)),-1))
median_cost_Album = filter_list(list(map(lambda x: x['trackPrice'], album_values)),-1)
median_cost_Album.sort()
median_cost_Album = statistics.median(median_cost_Album)

## Вывод данных по альбому

In [15]:
print('Album name:',album_name)

print('Count of songs:',album_songs_count)

print('Most popular genre count:','Rock -',dataGenre_Album.count('Rock'))

print('Album release date : ',*releaseDate_Album)

print('Songs Duration:',convert_millis(songsDuration_Album))

print('Average time:',convert_millis(Average_time_Album))

print('Median duration songs:',convert_millis(median_time_Album))

print('Longest song:', longest_song_Album['trackName'], convert_millis(longest_song_Album['trackTimeMillis']))

print('Shortest song:', shortest_song_Album['trackName'], convert_millis(shortest_song_Album['trackTimeMillis']))

print('Total cost of songs ≈ ',round(songsCost_Album),'$')

print('Average cost:',Average_cost_Album,'$')

print('Median cost songs:',median_cost_Album,'$')

print('The most expensive song:', dearest_song_Album['trackName'],'Trackrpice:',dearest_song_Album['trackPrice'])

if сheapest_song['trackPrice'] == -1 : 
    print('Сheapest song:', сheapest_song_Album['trackName'],'Track price:','Free')
else:
    print('Сheapest song:', сheapest_song_Album['trackName'],'Track price:',сheapest_song_Album['trackPrice'])

Album name: Десятилетие (Live at Adrenaline Stadium 2020)
Count of songs: 36
Most popular genre count: Rock - 36
Album release date :  2021-01-29T12:00:00Z
Songs Duration: 2:Hours,27:Minutes,39:Seconds
Average time: 0.0:Hours,4.0:Minutes,6.0:Seconds
Median duration songs: 0.0:Hours,3.0:Minutes,57.0:Seconds
Longest song: Лампами 0:Hours,8:Minutes,48:Seconds
Shortest song: Счастье 0:Hours,2:Minutes,0:Seconds
Total cost of songs ≈  36 $
Average cost: 0.99 $
Median cost songs: 0.99 $
The most expensive song: Будем друзьями Trackrpice: 0.99
Сheapest song: Будем друзьями Track price: 0.99
